<a href="https://colab.research.google.com/github/junior825/BERT/blob/main/tensorflow_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import urllib.request
import pandas as pd
import torch 
import numpy
# data 불러오기 => 네이버 평점에서 가져온 데이터
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
train_data = pd.read_table('ratings_train.txt')
train_data['document']

0                                       아 더빙.. 진짜 짜증나네요 목소리
1                         흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                             교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                  인간이 문제지.. 소는 뭔죄인가..
149996                                        평점이 너무 낮아서...
149997                      이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998                          청춘 영화의 최고봉.방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 150000, dtype: object

In [ ]:
# 해당 컴퓨터에 어떤 CPU/GPU가 있는지 확인해주는 작업
%tensorflow_version 2.x
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12316913417355973
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10843127808
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9086666038471237007
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
 xla_global_id: 416903419]

In [ ]:
train_data = pd.DataFrame(train_data) # 만약에 document나 Label에 전처리가 필요하다면 전처리 해야함 (LabelEncoding 등이 필요함)

train_data

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
train_data.groupby('label').size() # label size는 비슷하여 OverSampling / UnderSampling이 없는 것으로 파악할 수 있음

label
0    75173
1    74827
dtype: int64

In [ ]:
import re

# 특수문자를 제거하는 방법
def clean_text(inputString):
  text_rmv = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!·]', ' ', inputString) 
  text_rmv = ' '.join(text_rmv.split())
  return text_rmv

# List화 시키기
train_texts = train_data['document'].to_list()
train_labels = train_data['label'].to_list()

train_texts_2 = [clean_text(str(s)) for s in train_data['document']]

print(len(train_texts_2), len(train_labels))

150000 150000


In [ ]:
from sklearn.model_selection import train_test_split

# 데이터 분할을 통하여 학습하기
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts_2, train_labels, test_size=0.2, random_state=42)

In [ ]:
# !pip install transformers
from transformers import BertTokenizerFast

HUGGINGFACE_MODEL_PATH = "bert-base-multilingual-cased" # Hugging Face에 있는 모든 것들이 사용할 수 있는 것은 아니다.


tokenizer = BertTokenizerFast.from_pretrained(HUGGINGFACE_MODEL_PATH)
 # BertTokenizerFast는 [CLS] + [SEP] / Zero-Padding / Attention Mask 3가지른 모두 진행하주는 역할을 한다. => 따로 진행하게 될 경우 문제가 생길 수 있다.
 # BertTokenizerFast.from_pretrained 는 BERT로부터 pre-training 시킨 여러 모델들 중에서 한가지를 가져올 수 있다.

train_encodings = tokenizer(train_texts, truncation = True, padding = True) # truncation => 입력된 문장을 tokenizing했을 때 / padding => zero-padding을 할것인가 안할것인가
val_encodings = tokenizer(val_texts, truncation = True, padding = True)

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
print(train_encodings[1].tokens)
# print(train_encodings_2[1].tokens)
print(train_encodings[1])

['[CLS]', '배', '##비', '##장', '##전을', '독', '##특', '##하게', '각', '##색', '##한', '의', '##외', '##로', '재', '##미', '##있', '##게', '본', '영화', '배우', '##들의', '연', '##기', '##력', '##과', '사', '##극', '##톤', '##이', '부', '##족', '##한', '면', '##이', '아', '##쉽', '##지만', '격', '##려', '##의', '뜻', '##에서', '10', '##점', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PA

In [ ]:
import tensorflow as tf
# trainset-set
# tf.data.Dataset.from_tensor_slices => dict(train_encoding)과 train_labels를 새로운 데이터셋으로 변경하는 방법을 의미한다.

with tf.device('/device:GPU:0'):
  train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
    ))
  
  val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
    ))

In [ ]:
from transformers import TFBertForSequenceClassification

num_labels = len(train_data['label'].unique())
model = TFBertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels= num_labels , from_pt=True) # num_labels는 분류되는 정도가 얼마인지가 필요함

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy']) # 이중모형일 경우에는 loss를 binary_crossentropy / 다항일 경우에는 다른 crossentropy나 다른 것들로 해야됨

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

callback_earlystop = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001, # the threshold that triggers the termination (acc should at least improve 0.001)
    patience=2)

with tf.device('/device:GPU:0'): # device에 있는 GPU:0를 사용하겠다.
  model.fit(
    train_dataset.shuffle(1000).batch(16),
    validation_data=val_dataset.shuffle(1000).batch(16),
    epochs = 5, batch_size= 16,
    callbacks = [callback_earlystop]
    )
  
# hyperparameter => learning rate / optimizer / loss / epoch / batch_size 등을 조절하여야 한다. 

Epoch 1/5
3375/7500 [============>.................] - ETA: 58:17 - loss: 1.3566 - accuracy: 0.4928

KeyboardInterrupt: ignored